# OpenSearch 클러스터 생성

> 이 노트북은, SageMaker Notebook <i><b>conda_python3</b></i> 커널에서 테스트 되었습니다.

#### 중요
* OpenSearch 클러스터 생성으로 인한 "과금" 이 발생이 되는 부분 유념 해주시기 바랍니다.

In [1]:
#!pip install -r requirements.txt

## OpenSearch 도메인 생성 (콘솔에서 진행하는 경우)

아래는 OpenSearch 콘솔 화면에서 UI로 도메인을 생성하는 절차 입니다.
참고용으로만 봐주시고, 그림 설명 다음의 코드를 통해서 도메인을 생성합니다.

#### Step1. OpenSearch 콘솔로 이동 후, Navigator에서 Domain 이동 후 Create domain 선택
![aoss-01.png](./img/aoss-01.png)

#### Step2. Domain config 세팅
* Domain name :
* Domain creation Method: 사용자 지정생성 (손쉬운생성 선택시 '최대 절수' 오류 발생하는 경우)

![aoss-02.png](./img/aoss-02.png)

* Engine options: 2.11

* Network: Public access (실전에서는 VPC를 생성하여 VPC 액세스로 구성해야 합니다)

![aoss-03.png](./img/aoss-03.png)

* Master user: Create master user

* Master username, Master password and Confirm master password 입력

![aoss-04.png](./img/aoss-04.png)

* 고급클러스터 > 최대절수 선택 (손쉬운 생성 오류 경우)

![aoss-05.png](./img/aoss-05.png)

* 오른쪽 아래 주황색 create 선택

#### Step3. Access 설정
* 도메인 보안구성 > 편집 클릭

![aoss-06.png](./img/aoss-06.png)

* 도메인 수준 엑세스 정책 구성 > Effect : Allow 로 수정

![aoss-07.png](./img/aoss-07.png)


#### Step4. Domain enapoint 복사

![aoss-08.png](./img/aoss-08.png)

* [create_domain](https://boto3.amazonaws.com/v1/documentation/api/1.18.51/reference/services/opensearch.html#OpenSearchService.Client.create_domain)
* It takes about 20 mins

## OpenSearch 도메인 생성 (15~20분 소요)

<b>[ 중요 ]</b>
* SageMaker JupyterLab에서 아래 코드를 통해 OpenSearch Domain을 생성하는 경우, SageMaker Notebook IAM role에 OpenSearchFullAccess와 같은 권한이 필요합니다.

In [5]:
import boto3
import uuid
import botocore
import time
DEV = True # True일 경우 1-AZ without standby로 생성, False일 경우 3-AZ with standby. 워크샵 목적일 때는 지나친 과금/리소스 방지를 위해 True로 설정하는 것을 권장
VERSION = "2.11" # OpenSearch Version (예: 2.7 / 2.9 / 2.11)

opensearch_user_id = 'raguser'
opensearch_user_password = 'Passw0rd1!'

region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity()["Account"]
opensearch = boto3.client('opensearch', region)
rand_str = uuid.uuid4().hex[:8]
domain_name = f'rag-hol-{rand_str}'

cluster_config_prod = {
    'InstanceCount': 3,
    'InstanceType': 'r6g.large.search',
    'ZoneAwarenessEnabled': True,
    'DedicatedMasterEnabled': True,
    'MultiAZWithStandbyEnabled': True,
    'DedicatedMasterType': 'r6g.large.search',
    'DedicatedMasterCount': 3
}

cluster_config_dev = {
    'InstanceCount': 1,
    'InstanceType': 'r6g.large.search',
    'ZoneAwarenessEnabled': False,
    'DedicatedMasterEnabled': False,
}


ebs_options = {
    'EBSEnabled': True,
    'VolumeType': 'gp3',
    'VolumeSize': 100,
}

advanced_security_options = {
    'Enabled': True,
    'InternalUserDatabaseEnabled': True,
    'MasterUserOptions': {
        'MasterUserName': opensearch_user_id,
        'MasterUserPassword': opensearch_user_password
    }
}

ap = f'{{\"Version\":\"2012-10-17\",\"Statement\":[{{\"Effect\":\"Allow\",\"Principal\":{{\"AWS\":\"*\"}},\"Action\":\"es:*\",\"Resource\":\"arn:aws:es:{region}:{account_id}:domain\/{domain_name}\/*\"}}]}}'

if DEV:
    cluster_config = cluster_config_dev
else:
    cluster_config = cluster_config_prod
    
response = opensearch.create_domain(
    DomainName=domain_name,
    EngineVersion=f'OpenSearch_{VERSION}',
    ClusterConfig=cluster_config,
    AccessPolicies=ap,
    EBSOptions=ebs_options,
    AdvancedSecurityOptions=advanced_security_options,
    NodeToNodeEncryptionOptions={'Enabled': True},
    EncryptionAtRestOptions={'Enabled': True},
    DomainEndpointOptions={'EnforceHTTPS': True}
)

In [6]:
%%time
def wait_for_domain_creation(domain_name):
    try:
        response = opensearch.describe_domain(
            DomainName=domain_name
        )
        # Every 60 seconds, check whether the domain is processing.
        while 'Endpoint' not in response['DomainStatus']:
            print('Creating domain...')
            time.sleep(60)
            response = opensearch.describe_domain(
                DomainName=domain_name)

        # Once we exit the loop, the domain is ready for ingestion.
        endpoint = response['DomainStatus']['Endpoint']
        print('Domain endpoint ready to receive data: ' + endpoint)
    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'ResourceNotFoundException':
            print('Domain not found.')
        else:
            raise error

# OpenSearch 도메인 생성 - 약 20분 소요
wait_for_domain_creation(domain_name)

Creating domain...
Creating domain...
Creating domain...
Creating domain...
Creating domain...
Creating domain...
Creating domain...
Creating domain...
Creating domain...
Creating domain...
Creating domain...
Creating domain...
Creating domain...
Creating domain...
Creating domain...
Creating domain...
Domain endpoint ready to receive data: search-rag-hol-f5833a6c-oii6t4x4yosha7d62gjjelicca.us-west-2.es.amazonaws.com
CPU times: user 281 ms, sys: 7.53 ms, total: 289 ms
Wall time: 16min 3s


In [7]:
response = opensearch.describe_domain(DomainName=domain_name)
opensearch_domain_endpoint = f"https://{response['DomainStatus']['Endpoint']}"

# OpenSearch 도메인 Endpoint 확인
print(opensearch_domain_endpoint)

https://search-rag-hol-f5833a6c-oii6t4x4yosha7d62gjjelicca.us-west-2.es.amazonaws.com


---

## 한국어 처리를 위한 노리(Nori) 플러그인 설치

Amazon OpenSearch Service에서 유명한 오픈 소스 한국어 텍스트 분석기인 노리(Nori) 플러그인을 지원합니다. 기존에 지원하던 은전한닢(Seunjeon) 플러그인과 더불어 노리를 활용하면 개발자가 한국 문서에 대해 전문 검색을 쉽게 구현할 수 있습니다.

이와 함께, 중국어 분석을 위한 Pinyin 플러그인과 STConvert 플러그인, 그리고 일본어 분석을 위한 Sudachi 플러그인도 추가됐습니다. 노리 플러그인은 OpenSearch 1.0 이상 버전을 실행하는 신규 도메인과 기존 도메인에서 사용 가능합니다.

<b>[주의] 노리 플러그인 연동에는 약 25-27분의 시간이 소요됩니다.</b>

In [12]:
nori_pkg_id = {}
nori_pkg_id['us-east-1'] = {
    '2.3': 'G196105221',
    '2.5': 'G240285063',
    '2.7': 'G16029449', 
    '2.9': 'G60209291',
    '2.11': 'G181660338'
}

nori_pkg_id['us-west-2'] = {
    '2.3': 'G94047474',
    '2.5': 'G138227316',
    '2.7': 'G182407158', 
    '2.9': 'G226587000',
    '2.11': 'G79602591'
}

pkg_response = opensearch.associate_package(
    PackageID=nori_pkg_id[region][VERSION], # nori plugin
    DomainName=domain_name
)

In [13]:
%%time
def wait_for_associate_package(domain_name, max_results=1):

    response = opensearch.list_packages_for_domain(
        DomainName=domain_name,
        MaxResults=1
    )
    # Every 60 seconds, check whether the domain is processing.
    while response['DomainPackageDetailsList'][0]['DomainPackageStatus'] == "ASSOCIATING":
        print('Associating packages...')
        time.sleep(60)
        response = opensearch.list_packages_for_domain(
            DomainName=domain_name,
            MaxResults=1
        )

    #endpoint = response['DomainStatus']['Endpoint']
    print('Associated!')

wait_for_associate_package(domain_name)

Associating packages...
Associated!
CPU times: user 15.5 ms, sys: 4.39 ms, total: 19.8 ms
Wall time: 1min


opensearchpy를 이용하여 nori 플러그인 설치 여부를 확인합니다.

In [10]:
! pip list | grep langchain
! pip list | grep opensearch

langchain                     0.1.11
langchain-community           0.0.27
langchain-core                0.1.30
langchain-text-splitters      0.0.1
opensearch-dsl                2.1.0
opensearch-py                 2.4.2


아래의 실행 결과에서 "opensearch-nori plugin이 사용가능합니다." 라고 표시되면 노리 분석기가 사용 가능한 상태 입니다.<br>

In [14]:
from opensearchpy import OpenSearch, RequestsHttpConnection
http_auth = (opensearch_user_id, opensearch_user_password)
os_client = OpenSearch(
                hosts=[
                    {'host': opensearch_domain_endpoint.replace("https://", ""),
                     'port': 443
                    }
                ],
                http_auth=http_auth, # Master username, Master password,
                use_ssl=True,
                verify_certs=True,
                connection_class=RequestsHttpConnection
            )

res_str = os_client.cat.plugins()

if 'opensearch-analysis-nori' in res_str:
    print('opensearch-nori plugin이 사용가능합니다.')
else:
    print('opensearch-nori plugin 연결이 진행되지 않았습니다.')

opensearch-nori plugin 연결이 진행되지 않았습니다.


In [17]:
# 다음 노트북에서 OpenSearch 연결 정보를 활용하기 위해 변수 저장

%store opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint

Stored 'opensearch_user_id' (str)
Stored 'opensearch_user_password' (str)
Stored 'domain_name' (str)
Stored 'opensearch_domain_endpoint' (str)


## (필요시) Clean-up : OpenSearch 도메인 삭제

비용 발생을 막기 위해 OpenSearch를 사용하지 않는다면 아래의 명령어로 도메인을 삭제 합니다.<br>
(OpenSearch 콘솔에서 직접 생성한 도메인을 선택하고 Delete를 하셔도 됩니다)

In [ ]:
import boto3

client = boto3.client('opensearch')

response = client.delete_domain(
    DomainName=opnsearch_config["domain"]
)